# Produce random files for given nuclide and library

In [ ]:
import sandy

In this examples we'll produce random ACE files for the JEFF-3.3 Pu-238 evaluation.
This sequence can we further extended for other nuclides, individually or together in a loop.

In [ ]:
lib_name = "jeff_33"
nucl = 942390
filename = f"{nucl}.{lib_name}"

In [ ]:
endf6 = sandy.get_endf6_file(lib_name, 'xs', nucl)
endf6.to_file(filename)

In [ ]:
nsmp = 2  # number of samples
processes = 1  # change if you want to run in parallel (better on Linux)

In [ ]:
# command line
cli = f"{filename}  --processes {processes}  --samples {nsmp}  --mf 33  --temperatures 300  --acer  --debug"
sandy.sampling.run(cli.split())

This is equivalent to the command line entry
```bash
python -m sandy.sampling  942390.jeff_33  --processes 1  --samples 2  --temperatures 300  --acer  --debug
```

> Notice that option `--debug` should be removed for production.

If you need more control over your pipeline you can generate samples using the python API.

Some of the reasons identified to do this are:
- have access to the perturbation coefficients
- change the parameter `err` (used for by the NJOY modules RECONR and BROADR) to make the code run faster
- remove some NJOY modules such as PURR
- apply perturbations to already Doppler-broadenened cross sections (more details below)

This is equivalent to what was previously run.

In [ ]:
# this generates samples for cross sections and nubar
smps = endf6.get_perturbations(
    nsmp,
    njoy_kws=dict(
        err=1,   # very fast calculation, for testing
        chi=False,
        mubar=False,
        xs=True,
        nubar=False,
        verbose=True,
    ),
)

In [ ]:
# This part applies the perturbations (samples) to the best-estimates and writes perturbed output files samples for cross sections and nubar
outs = endf6.apply_perturbations(
    smps,
    processes=processes,
    njoy_kws=dict(err=1),   # very fast calculation, for testing
    to_ace=True,   # produce ACE files
    to_file=True,
    ace_kws=dict(err=1, temperature=300, verbose=True, purr=False, heatr=False, thermr=False, gaspr=False),
    verbose=True,
)